In [ ]:
import requests
import pandas as pd
from datetime import datetime
from math import log, sqrt, exp
from scipy.stats import norm
from scipy.optimize import brentq
import yfinance as yf
import time
from typing import Optional

# ----------------------------
# 1. Black-Scholes IV Function
# ----------------------------

def black_scholes_price(S, K, T, r, sigma, option_type="call"):
    """
    Calculates the theoretical price of a European option using the Black-Scholes model.

    Args:
        S (float): Current price of the underlying asset.
        K (float): Strike price of the option.
        T (float): Time to expiration in years (e.g., 0.5 for 6 months).
        r (float): Risk-free interest rate (annualized, e.g., 0.05 for 5%).
        sigma (float): Volatility of the underlying asset (annualized).
        option_type (str): Type of the option, either "call" or "put".

    Returns:
        float: The theoretical Black-Scholes option price.
    """
    # Calculate d1 parameter of the Black-Scholes formula.
    # d1 incorporates the asset price, strike price, time, risk-free rate, and volatility.
    d1 = (log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrt(T))
    # Calculate d2 parameter, which is derived from d1.
    d2 = d1 - sigma * sqrt(T)

    # Calculate option price based on option type.
    if option_type == "call":
        # Black-Scholes formula for a call option.
        # norm.cdf(d1) is the cumulative standard normal distribution function of d1.
        return S * norm.cdf(d1) - K * exp(-r * T) * norm.cdf(d2)
    else:
        # Black-Scholes formula for a put option.
        return K * exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

def implied_volatility(price, S, K, T, r, option_type="call"):
    """
    Calculates the implied volatility (IV) of an option using the Black-Scholes model
    and Brent's method to find the root.

    Implied volatility is the volatility that, when input into the Black-Scholes formula,
    yields the current market price of the option.

    Args:
        price (float): The current market price of the option.
        S (float): Current price of the underlying asset.
        K (float): Strike price of the option.
        T (float): Time to expiration in years.
        r (float): Risk-free interest rate (annualized).
        option_type (str): Type of the option, either "call" or "put".

    Returns:
        float or None: The implied volatility, or None if calculation fails (e.g., no real root found).
    """
    try:
        # brentq is a root-finding algorithm. It tries to find a sigma value
        # such that black_scholes_price(S, K, T, r, sigma, option_type) - price = 0.
        # The search range for sigma is typically between a very small positive number (1e-6)
        # and a reasonably large value (5.0, representing 500% volatility).
        return brentq(lambda sigma: black_scholes_price(S, K, T, r, sigma, option_type) - price, 1e-6, 5)
    except ValueError:
        # If brentq cannot find a root within the given interval, a ValueError is raised.
        # This can happen if the market price is inconsistent with the Black-Scholes model
        # for any realistic volatility.
        return None

# ----------------------------
# 2. Get Call Options from FMP
# ----------------------------

def get_call_options_fmp(ticker, api_key):
    """
    Fetches call option chain data for a given ticker from the Financial Modeling Prep (FMP) API.

    Args:
        ticker (str): The stock ticker symbol (e.g., "AAPL").
        api_key (str): Your FMP API key.

    Returns:
        list: A list of dictionaries, where each dictionary represents a call option contract.

    Raises:
        Exception: If the API request fails (e.g., non-200 status code).
    """
    # Construct the API URL for the options chain.
    url = f"https://financialmodelingprep.com/api/v3/options-chain/{ticker}?apikey={api_key}"
    print(url)  # Print the URL for debugging purposes.
    # Send a GET request to the FMP API.
    response = requests.get(url)
    # Check if the HTTP request was successful (status code 200).
    if response.status_code != 200:
        # Raise an exception if the API returns an error.
        raise Exception(f"API Error: {response.status_code}")
    # Parse the JSON response into a Python dictionary/list.
    options_data = response.json()
    # Filter the data to include only call options and return the list.
    return [opt for opt in options_data if opt['contractType'] == 'CALL']

# ----------------------------
# 2. Get Call Options from yfinance
# ----------------------------

def get_call_options(
    ticker_symbol: str, 
    strike_price: float, 
    retries: int = 5, 
    delay: int = 2
) -> Optional[pd.DataFrame]:
    """
    Retrieves call options for a given ticker and strike price from Yahoo Finance.

    This function includes a retry mechanism with exponential backoff to mitigate
    potential rate-limiting errors from the Yahoo Finance API.

    Args:
        ticker_symbol (str): The stock ticker symbol (e.g., 'AAPL', 'MSFT').
        strike_price (float): The specific strike price to filter for.
        retries (int): The maximum number of retry attempts for a failed API call.
        delay (int): The initial delay (in seconds) between retries. This delay
                     will double after each failed attempt (exponential backoff).

    Returns:
        Optional[pd.DataFrame]: A pandas DataFrame containing all matching call
        options across all expiration dates. The DataFrame includes the current
        spot price for context. Returns an empty DataFrame if no options match
        the criteria. Returns None if the ticker is invalid or all API requests fail.
    """
    
    # Note: We fetch the spot price directly instead of taking it as an argument.
    # This ensures we are comparing the strike to the most current market price
    # available from the data source, avoiding potential discrepancies.

    current_delay = delay
    for i in range(retries):
        try:
            # 1. Create a Ticker object
            ticker = yf.Ticker(ticker_symbol)
            
            # Fetch basic info. This call can sometimes fail under heavy load.
            info = ticker.info
            spot_price = info.get('regularMarketPrice')
            
            if not spot_price:
                # Fallback to last closing price if live market price is not available
                hist = ticker.history(period="1d")
                if not hist.empty:
                    spot_price = hist['Close'].iloc[-1]
                else: # If all fails
                    print(f"Warning: Could not determine spot price for {ticker_symbol}.")
                    spot_price = 0.0

            # 2. Get all available expiration dates for the options
            expirations = ticker.options
            if not expirations:
                print(f"Warning: No options expiration dates found for {ticker_symbol}.")
                return pd.DataFrame() # Return empty DataFrame as there are no options

            all_matching_calls = []

            # 3. Loop through each expiration date to get the option chain
            for exp_date in expirations:
                # Fetch the option chain for the specific expiration date
                # This is another API call that could potentially fail.
                opt_chain = ticker.option_chain(exp_date)
                
                # Filter the calls DataFrame for the specified strike price
                calls = opt_chain.calls
                matching_strike = calls[calls['strike'] == strike_price]
                
                if not matching_strike.empty:
                    all_matching_calls.append(matching_strike)
            
            # 4. Consolidate results and return
            if not all_matching_calls:
                print(f"No call options found for {ticker_symbol} with a strike price of ${strike_price:.2f}.")
                return pd.DataFrame()

            # Concatenate all found options into a single DataFrame
            result_df = pd.concat(all_matching_calls)
            
            # Add the fetched spot price to each row for easy reference
            result_df['spotPrice'] = spot_price
            
            # Reorder columns to have key info first
            cols_to_front = ['contractSymbol', 'lastTradeDate', 'strike', 'lastPrice', 'bid', 'ask', 'change', 'volume', 'openInterest', 'impliedVolatility', 'spotPrice']
            # Ensure all desired columns exist before reordering
            existing_cols = [col for col in cols_to_front if col in result_df.columns]
            other_cols = [col for col in result_df.columns if col not in existing_cols]
            result_df = result_df[existing_cols + other_cols]

            return result_df

        except Exception as e:
            print(f"Attempt {i + 1}/{retries} failed for {ticker_symbol}: {e}")
            if i < retries - 1:
                print(f"Retrying in {current_delay} seconds...")
                time.sleep(current_delay)
                current_delay *= 2  # Exponential backoff
            else:
                print(f"All {retries} attempts failed for {ticker_symbol}. Aborting.")
                return None
    
    return None # Should be unreachable, but as a fallback.


    

# ----------------------------
# 3. Calculate IV by Expiration
# ----------------------------

def calculate_iv_curve(ticker, spot_price, risk_free_rate, api_key):
    """
    Calculates the implied volatility for all available call options for a given ticker
    and returns a DataFrame with option details and their calculated IVs.

    Args:
        ticker (str): The stock ticker symbol.
        spot_price (float): The current market price of the underlying stock.
        risk_free_rate (float): The annualized risk-free interest rate (e.g., 0.05).
        api_key (str): Your FMP API key.

    Returns:
        pd.DataFrame: A DataFrame containing 'expirationDate', 'strike', 'lastPrice',
                      and 'impliedVolatility' for each call option, or an empty DataFrame
                      if no valid options are found.
    """
    # Fetch all call options for the specified ticker.
    calls = get_call_options(ticker, spot_price)
    print(f"Number of call options fetched for {ticker}: {len(calls)}")
    # Initialize an empty list to store dictionaries of IV data.
    iv_data = []

    # Get today's date to calculate time to expiration.
    today = datetime.today()

    # Iterate through each call option fetched from the API.
    for call in calls:
        # Extract relevant option details.
        strike = call['strike']
        last_price = call['lastPrice']
        expiration_str = call['expirationDate'] # Expiration date as a string (e.g., "YYYY-MM-DD").

        try:
            # Convert the expiration date string to a datetime object.
            expiration = datetime.strptime(expiration_str, "%Y-%m-%d")
            # Calculate Time to Expiration (T) in years.
            # (expiration - today).days gives the number of days difference.
            # Divide by 365.0 to convert days into years.
            T = (expiration - today).days / 365.0
            # Skip options that have already expired or have zero time to expiration.
            if T <= 0:
                continue
            # Calculate the implied volatility for the current option.
            iv = implied_volatility(last_price, spot_price, strike, T, risk_free_rate, option_type="call")
            # If a valid implied volatility is returned (not None), append it to the data list.
            if iv:
                iv_data.append({
                    "expirationDate": expiration_str,
                    "strike": strike,
                    "lastPrice": last_price,
                    "impliedVolatility": round(iv, 4) # Round IV to 4 decimal places for readability.
                })
        except Exception as e:
            # Catch any exceptions that might occur during date parsing or IV calculation
            # (e.g., malformed date string, issues with implied_volatility).
            # Print the error for debugging, but continue processing other options.
            # print(f"Error processing option: {call}. Error: {e}") # Uncomment for debugging
            continue # Skip to the next option if an error occurs.

    # Convert the list of IV data dictionaries into a pandas DataFrame.
    return pd.DataFrame(iv_data)

# ----------------------------
# 4. Run Example
# ----------------------------

# This block ensures that the code inside it only runs when the script is executed directly,
# not when it's imported as a module into another script.
if __name__ == "__main__":
    # --- Configuration Parameters ---
    # IMPORTANT: Replace "YOUR_FMP_API_KEY" with your actual Financial Modeling Prep API key.
    api_key = "v0Y7rqjEfz0nBiixKBqJwLLgyFYbOUGA"
    # The stock ticker symbol for which to calculate IV.
    ticker = "AAPL"
    # The current spot price of the underlying stock.
    # In a real-world application, this would be fetched dynamically from a live market data source.
    spot_price = 190.0
    # The risk-free interest rate, expressed as a decimal (e.g., 0.05 for 5%).
    # This is typically approximated using the yield of a short-term government bond.
    risk_free_rate = 0.05

    # Call the main function to calculate the implied volatility curve.
    iv_df = calculate_iv_curve(ticker, spot_price, risk_free_rate, api_key)

    # --- Data Aggregation and Display ---
    # Group the DataFrame by 'expirationDate' and calculate the mean (average)
    # implied volatility for each expiration.
    grouped = iv_df.groupby("expirationDate")["impliedVolatility"].mean().reset_index()
    # Rename the columns for clarity: 'date' for expiration date and 'avgImpliedVolatility'.
    grouped.columns = ["date", "avgImpliedVolatility"]

    # Print the resulting DataFrame, showing the average implied volatility for each expiration date.
    print(grouped)

https://financialmodelingprep.com/api/v3/options-chain/AAPL?apikey=v0Y7rqjEfz0nBiixKBqJwLLgyFYbOUGA
Number of call options fetched for AAPL: 0


KeyError: 'expirationDate'